In [ ]:
#importing the libraries
# from urllib.request import urlopen
# import requests 
import re
import pandas as pd
# import numpy as np
# from collections import defaultdict
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

In [ ]:
driver = webdriver.Chrome()

# Logging into LinkedIn
driver.get("https://linkedin.com/uas/login")
time.sleep(5)

username = driver.find_element(By.ID, "username")
username.send_keys("anrs0310@proton.me") # Enter Your Email Address

pword = driver.find_element(By.ID, "password")
pword.send_keys("SrqoBtP2")	 # Enter Your Password

driver.find_element(By.XPATH, "//button[@type='submit']").click()


In [ ]:
exceldata = pd.read_csv("udemy-datacleaning.csv")
LinkedIn_company_url = []
Index_company_datas = []
LinkedIn_personal_url = []
Index_personal_datas = []

for idx in exceldata.index:
    linkedin = exceldata['linkedin_url'][idx]
    Index_data = exceldata['index_number'][idx]

    to_check = "company"

    if re.search(r'\b' + re.escape(to_check) + r'\b', linkedin):
        LinkedIn_company_url.append(linkedin)
        Index_company_datas.append(Index_data)
    else:
        LinkedIn_personal_url.append(linkedin)
        Index_personal_datas.append(Index_data)

print(f"Number of company urls = {len(LinkedIn_company_url)}")
print(f"Number of personal urls = {len(LinkedIn_personal_url)}")



# Code for scraping personal account

In [ ]:
Index_personal = []
Instructor_LinkedIn_url = []
Instructor_name = []
Instructor_worksat = []
Instructor_followers = []
Instructor_connections = []
About_Instructor = []
Post1_frequency = []
Post1_numberof_reactions = []
Post1_numberof_comments = []
Post2_frequency = []
Post2_numberof_reactions = []
Post2_numberof_comments = []
Post3_frequency = []
Post3_numberof_reactions = []
Post3_numberof_comments = []


durations = [[],[],[]]
reactions = [[],[],[]]
comments = [[],[],[]]


In [ ]:

# No Post : https://linkedin.com/in/pianorosa
#profile_url = ["https://www.linkedin.com/in/philip-ebiner-06728413/","https://linkedin.com/in/vishukamble","https://linkedin.com/in/svekis","https://www.linkedin.com/in/gilseone/"]

count1 = 0


# id 427 = 'https://linkedin.com/profile/preview?locale=en_US&trk=prof-0-sb-preview-primary-button' it will be empty set 

exclusion_set = {'https://linkedin.com/profile/preview?locale=en_US&trk=prof-0-sb-preview-primary-button'} 

for id,p_url in zip(Index_personal_datas,LinkedIn_personal_url):

    count1 = count1 + 1
    print(f'{count1=} {id=} {p_url=}')

    if p_url in exclusion_set:
        continue

    try:

        Index_personal.append(id)

        Instructor_LinkedIn_url.append(p_url)

        driver.get(p_url)	 # this will open the link

        # https://www.selenium.dev/documentation/webdriver/waits/
        # driver.implicitly_wait(5)

        time.sleep(6)

        # Extracting the HTML of the complete introduction box
        # that contains the name, company name, and the location
        src = driver.page_source
        
        # Now using beautiful soup
        soup = BeautifulSoup(src, 'lxml')
        # time.sleep(15)

        error_404 = soup.find('body', {'id': 'error404'})
        page_not_found = soup.find('div', {'class': 'page-not-found'})
        intro = soup.find('div', {'class': 'pv-text-details__left-panel'})

        if error_404 or page_not_found or not intro:
            print("404")
            Instructor_name.append("N/A")
            Instructor_worksat.append("N/A")
            Instructor_connections.append("N/A")
            Instructor_followers.append("N/A")
            About_Instructor.append("N/A")
            for i in range(3):
                durations[i].append("N/A")
                reactions[i].append("N/A")
                comments[i].append("N/A")
            continue
        

        name_loc = intro.find("h1")
        if name_loc:
            name = name_loc.get_text().strip()
            Instructor_name.append(name)
        else:
            Instructor_name.append("")


        works_at_loc = intro.find("div", {'class': 'text-body-medium'})
        if works_at_loc:
            works_at = works_at_loc.get_text().strip()
            Instructor_worksat.append(works_at)
        else:
            Instructor_worksat.append("")

        
        about_section = soup.find('div',{'class': 'pv-shared-text-with-see-more full-width t-14 t-normal t-black display-flex align-items-center'})
        if about_section:
            absec = about_section.get_text().strip()
            About_Instructor.append(absec)
        else:
            About_Instructor.append("")


        
        connections = soup.find('ul', {'class': 'pv-top-card--list pv-top-card--list-bullet'}).find('span',{'class' :'t-black--light'})
        if connections:
            connect = connections.get_text().strip()
            Instructor_connections.append(connect)
        else:   
            Instructor_connections.append("")


        
        followers_count = soup.find('p', {'class': 'pvs-header__subtitle pvs-header__optional-link text-body-small'}).find('span',{'class':'visually-hidden'})
        if followers_count:
            fc = followers_count.get_text().strip()
            Instructor_followers.append(fc)
        else:
            Instructor_followers.append("")


        posts = soup.find('ul', {'class': 'display-flex flex-wrap list-style-none justify-space-between'})
        if not posts:
            for i in range(3):
                durations[i].append("N/A")
                reactions[i].append("N/A")
                comments[i].append("N/A")
            continue
            
        post1 = posts.findAll('li', {'class': 'profile-creator-shared-feed-update__mini-container'})

        for i in range(3): # post1, post2, post3

            if i > len(post1)-1:
                durations[i].append("")
                reactions[i].append("")
                comments[i].append("")
                continue
            element = post1[i]

            post_duration_data = element.find('span', {'class': 'feed-mini-update-contextual-description__text'})
            if post_duration_data:
                # Find the nested span with class 'visually-hidden' and extract its text
                post_duration_text = post_duration_data.find("span", {"class": "visually-hidden"}).text.strip()
                durations[i].append(post_duration_text)
            else:
                durations[i].append("")

            reaction_data = element.find('span', {'class' : 'social-details-social-counts__reactions-count' })
            if reaction_data:
                # Find the nested span with class 'visually-hidden' and extract its text
                reaction_text = reaction_data.get_text().strip()
                reactions[i].append(reaction_text)
            else:    
                reactions[i].append("")


            comments_data = element.find('li', {'class' : 'social-details-social-counts__item social-details-social-counts__comments social-details-social-counts__item--with-social-proof'})
            if comments_data:
                # Find the nested span with class 'visually-hidden' and extract its text
                comment_text = comments_data.get_text(strip=True)  # Get the text and remove leading/trailing whitespace
                comments[i].append(comment_text)
            else:
                comments[i].append("")
        
        time.sleep(20)

    except Exception as e:
        print("Timeout occurred")

        print(e)


Post1_frequency= durations[0]
Post1_numberof_reactions = reactions[0]
Post1_numberof_comments = comments[0]

Post2_frequency= durations[1]
Post2_numberof_reactions = reactions[1]
Post2_numberof_comments = comments[1]

Post3_frequency= durations[2]
Post3_numberof_reactions = reactions[2]
Post3_numberof_comments = comments[2]

In [ ]:
data1 = {'Index' : Index_personal,
        'LinkedIn Url' : Instructor_LinkedIn_url,
    'Instructor Name' : Instructor_name,
'Instructor work status' : Instructor_worksat,
'Number of followers' : Instructor_followers,
'Number of connections' : Instructor_connections,
'About section' : About_Instructor,
'Frequency of the latest post' : Post1_frequency,
'Number of reactions got for latest post' : Post1_numberof_reactions,
'Number of comments got for latest post' : Post1_numberof_comments,
'Frequency of the second latest post' : Post2_frequency,
'Number of reactions got for second latest post' : Post2_numberof_reactions,
'Number of comments got for second latest post' : Post2_numberof_comments,
'Frequency of the third latest post' : Post3_frequency,
'Number of reactions got for third latest post' : Post3_numberof_reactions,
'Number of comments got for third latest post' : Post3_numberof_comments }

li_per_df = pd.DataFrame(data1)
li_per_df

In [ ]:
li_per_df.to_csv('LinkedIn_instructor_data7.csv')

In [ ]:
df3 = pd.read_csv('LinkedIn_instructor_data123456.csv')
df4 = pd.read_csv('LinkedIn_instructor_data7.csv')

In [ ]:
appended_df2 = pd.concat([df3, df4], ignore_index=True)

appended_df2.to_csv('LinkedIn_instructor_data.csv')

In [ ]:
personal_data = pd.read_csv('LinkedIn_instructor_data.csv')
personal_data


# Code for scraping company pages

In [ ]:
Index_company = []
Company_LinkedIn_url = []
Company_name = []
Company_worksat = []
Company_followers = []
Number_of_employees = []
About_Company = []
CPost1_frequency = []
CPost1_numberof_reactions = []
CPost1_numberof_comments = []
CPost2_frequency = []
CPost2_numberof_reactions = []
CPost2_numberof_comments = []


In [ ]:


count2 = 0

for id,p_url in zip(Index_company_datas,LinkedIn_company_url):

    count2 = count2 + 1
    print(f'{count2=} {id=} {p_url=}')


    try:

        Index_company.append(id)

        Company_LinkedIn_url.append(p_url)

        driver.get(p_url)	 # this will open the link

        # https://www.selenium.dev/documentation/webdriver/waits/
        # driver.implicitly_wait(5)

        time.sleep(6)

        # Extracting the HTML of the complete introduction box
        # that contains the name, company name, and the location
        src = driver.page_source
        
        # Now using beautiful soup
        soup = BeautifulSoup(src, 'lxml')
        # time.sleep(15)

        error_404 = soup.find('body', {'id': 'error404'})
        page_not_found = soup.find('div', {'class': 'page-not-found'})
        intro = soup.find('div', {'class': 'ph5 pb5'})

        if error_404 or page_not_found or not intro:
            print("404")
            Company_name.append("N/A")
            Company_worksat.append("N/A")
            Company_followers.append("N/A")
            Number_of_employees.append("N/A")
            About_Company.append("N/A")
            CPost1_frequency.append("N/A")
            CPost1_numberof_reactions.append("N/A")
            CPost1_numberof_comments.append("N/A")
            CPost2_frequency.append("N/A")
            CPost2_numberof_reactions.append("N/A")
            CPost2_numberof_comments.append("N/A")
            continue
                    

       
        comp_name = intro.find("h1", {'class': 'ember-view text-display-medium-bold org-top-card-summary__title full-width'})
        if comp_name:
            comp_name_text = comp_name.get_text().strip()
            Company_name.append(comp_name_text)
        else:
            Company_name.append("")
        

        work_status = intro.find("p", {'class': 'org-top-card-summary__tagline'})
        if work_status:
            ws_text = work_status.get_text().strip()
            Company_worksat.append(ws_text)
        else:
            Company_worksat.append("")

    
        followers_text = intro.findAll("div", {'class': 'org-top-card-summary-info-list__info-item'})
        if followers_text:
            fc_text = ""
            for ft in followers_text:
                if "followers" in ft.get_text().strip():
                    fc_text = ft.get_text().strip()
            Company_followers.append(fc_text)
        else:
             Company_followers.append("")


        employee = intro.find("span", {'class': 't-normal t-black--light link-without-visited-state link-without-hover-state'})
        if employee:
            emp_text = employee.get_text().strip()
            Number_of_employees.append(emp_text)
        else:
            Number_of_employees.append("")

        about_section = soup.find('div',{'class': 'ember-view organization-about-module__content-consistant-cards-description'})
        if about_section:
            abtsec_text = about_section.get_text().strip('\n')
            About_Company.append(abtsec_text)
        else:
            About_Company.append("")


        posts = soup.find('ul', {'class': 'artdeco-carousel__slider ember-view'})

        if not posts:
            CPost1_frequency.append("N/A")
            CPost1_numberof_reactions.append("N/A")
            CPost1_numberof_comments.append("N/A")
            CPost2_frequency.append("N/A")
            CPost2_numberof_reactions.append("N/A")
            CPost2_numberof_comments.append("N/A")
            continue
        
        try:

            post1_data = posts.find('li', {'class': 'artdeco-carousel__item active ember-view'})
        except:
            CPost1_frequency.append("")
            CPost1_numberof_reactions.append("")
            CPost1_numberof_comments.append("")

        try:
            post1_duration_data = post1_data.find('div', {'class': "update-components-text-view break-words"}).get_text().strip()
            CPost1_frequency.append(post1_duration_data)
        except:
            CPost1_frequency.append("")

        try:
            post1_reaction_data = post1_data.find('span', {'class': "social-details-social-counts__reactions-count"}).get_text().strip()
            CPost1_numberof_reactions.append(post1_reaction_data)
        except:
            CPost1_numberof_reactions.append("")
        
        try:
            post1_comments_data = post1_data.find('li', {'class': "social-details-social-counts__item social-details-social-counts__comments"}).get_text().strip()
            CPost1_numberof_comments.append(post1_comments_data)
        except:
            CPost1_numberof_comments.append("")

        try:

            post2_data = posts.find('li', {'class': 'artdeco-carousel__item ember-view'})
        except:
            CPost2_frequency.append("")
            CPost2_numberof_reactions.append("")
            CPost2_numberof_comments.append("")


        try:
            post2_duration_data = post2_data.find('div', {'class': "update-components-text-view break-words"}).get_text().strip()
            CPost2_frequency.append(post2_duration_data)
        except:
            CPost2_frequency.append("")

        try:
            post2_reaction_data = post2_data.find('span', {'class': "social-details-social-counts__reactions-count"}).get_text().strip()
            CPost2_numberof_reactions.append(post2_reaction_data)
        except:
            CPost2_numberof_reactions.append("")
        
        try:
            post2_comments_data = post2_data.find('li', {'class': "social-details-social-counts__item social-details-social-counts__comments"}).get_text().strip()
            CPost2_numberof_comments.append(post2_comments_data)
        except:
            CPost2_numberof_comments.append("")
               
        time.sleep(20)

    except Exception as e:
        print("Timeout occurred")

        print(e)



id=42 p_url='https://linkedin.com/company/tokenmeister/'
id=45 p_url='https://linkedin.com/company/stone-river-elearning'
id=57 p_url='https://linkedin.com/company/packt-publishing'
count2=281 id=1755 p_url='https://linkedin.com/company/start-tech-academy'
count2=282 id=1763 p_url='https://linkedin.com/company/youaccel/'


In [ ]:
data2 = {'Index' : Index_company,
        'LinkedIn Url' : Company_LinkedIn_url,
    'Company Name' : Company_name,
'Company work status' : Company_worksat,
'Number of followers' : Company_followers,
'Number of employees' : Number_of_employees,
'About section' : About_Company,
'Frequency of the latest post' : CPost1_frequency,
'Number of reactions got for latest post' : CPost1_numberof_reactions,
'Number of comments got for latest post' : CPost1_numberof_comments,
'Frequency of the second latest post' : CPost2_frequency,
'Number of reactions got for second latest post' : CPost2_numberof_reactions,
'Number of comments got for second latest post' : CPost2_numberof_comments
 }

li_comp_df = pd.DataFrame(data2)
li_comp_df

In [ ]:
li_comp_df.to_csv('LinkedIn_company_data.csv')